<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/Sin-Dientes/proyecto_bbva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Armado del entorno

In [ ]:
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [ ]:
# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [ ]:
!readlink -f $(which java) | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
# Set up the environment for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [ ]:
# Install library for finding Spark
!pip install -q findspark

# Import the libary
import findspark

# Initiate findspark
findspark.init()

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()

# Check Spark Session Information
spark

# Importado de funciones

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, concat_ws, col, row_number, desc, collect_list, to_json, struct, year, current_date, datediff, floor, when, lit
from pyspark.sql.window import Window

In [ ]:
#Creación de un SQL context
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Carga de parquets

In [ ]:
df_customer = spark.read.parquet('/content/Datasets/customer_basics.snappy.parquet')
df_phones = spark.read.parquet('/content/Datasets/phones.snappy.parquet')
df_address = spark.read.parquet('/content/Datasets/address.snappy.parquet')
df_emails = spark.read.parquet('/content/Datasets/emails.snappy.parquet')
df_marital_status = spark.read.parquet('/content/Datasets/marital_status_type.parquet')
df_segment_type = spark.read.parquet('/content/Datasets/segment_type.parquet')
df_customer_documents = spark.read.parquet('/content/Datasets/customer_documents.parquet')
df_address_type = spark.read.parquet('/content/Datasets/address_type.parquet')
df_gender = spark.read.parquet('/content/Datasets/gender.parquet')
df_nationality = spark.read.parquet('/content/Datasets/nationality.parquet')
df_personal_type = spark.read.parquet('/content/Datasets/personal_type.parquet')
df_phone_type = spark.read.parquet('/content/Datasets/phone_type.parquet')
df_province = spark.read.parquet('/content/Datasets/province.parquet')
df_customer_segment = spark.read.parquet('/content/Datasets/customer_segment.parquet')
df_customer_info_temp = spark.read.parquet('/content/Datasets/customer_info_temp.parquet')

### Normalización de datos

In [ ]:
df_customer = df_customer.withColumnRenamed('last_change_date', 'l_c_d_customer')

In [ ]:
df_customer = df_customer.withColumn("job_type_desc", 
                                             when(df_customer.job_type == 1, 'REL.DEPENDENCIA/FIJO')
                                             .when(df_customer.job_type == 2, 'TEMPORAL')
                                             .when(df_customer.job_type == 3, 'AUTONOMO/INDEPENDIENTE')
                                             .when(df_customer.job_type == 3, 'OTROS')
                                             .otherwise("-"))

In [ ]:
df_marital_status_ok = df_marital_status.withColumn("marital_status_type", 
                                             when(df_marital_status.martial_status_short_desc == "NO INFORMA", 0)
                                             .when(df_marital_status.martial_status_short_desc == "CASADO/A  ", 1)
                                             .when(df_marital_status.martial_status_short_desc == "VIUDO/A   ", 2)
                                             .when(df_marital_status.martial_status_short_desc == "SEPARADO/A", 3)
                                             .when(df_marital_status.martial_status_short_desc == "DIVORCIADO", 4)
                                             .when(df_marital_status.martial_status_short_desc == "SOLTERO/A ", 5)
                                             .when(df_marital_status.martial_status_short_desc == "CONVIVIENT", 6)
                                             .when(df_marital_status.martial_status_short_desc == "OTROS     ", 7)
                                             .when(df_marital_status.martial_status_short_desc == "CONV. INSC", 8)
                                             .otherwise("-"))

# Extraccion de datos



### - 24 - Extraccion de datos desde parquet, clientes y teléfonos.
####Crear un DataFrame que contenga el JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_phones.

In [ ]:
df_customer_phones = df_phones.join(df_customer, 'customer_id')

In [ ]:
df_customer_phones.show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+-------------+---------------+-----------------------+------------------+---------------------+-----------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+----------------

### - 25 - Extraccion de datos desde .parquet, clientes y direcciones
####Crear un DataFrame que contenga el JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_adress.

In [ ]:
df_customer_address = df_address.join(df_customer, 'customer_id')

In [ ]:
df_customer_address.show()

+-----------+-----------------------+-------------------+--------------+--------------------+------------------+---------------------------+-----------------+-------------+---------------------+--------------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+-------------+---

### - 26 - Extraccion de datos desde .parquet, clientes y correos electrónicos
#### Crear un DataFrame que contenga el JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_email.   

In [ ]:
df_customer_emails = df_emails.join(df_customer, 'customer_id')

In [ ]:
df_customer_emails.show()

+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+--------------------------+--------------+----------------+-------------------+--------------------+-----------------------+---------------------+--------------------------+-------------------+----------------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+------------------+----------------+-----------------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+---

# Filtrar datos, reducir volumen

####- 27 - Filtrar el DataFrame de contactos telefónicos de clientes y resguardar los 3 contactos más actuales por cliente.

In [ ]:
df_customer_phones_sorted = df_customer_phones.orderBy([df_customer_phones.customer_id, desc('last_change_date')])

In [ ]:
window = Window.partitionBy(df_customer_phones_sorted.customer_id).orderBy(desc(df_customer_phones_sorted.last_change_date))

In [ ]:
df_phone_contact = df_customer_phones_sorted.withColumn('row_num', row_number().over(window))
df_phone_contact = df_phone_contact.filter(df_phone_contact.row_num <= 3)

In [ ]:
df_phone_contact = df_phone_contact.withColumn("full_phone", concat_ws("-", 'prefix_phone_id', 'phone_area_id', 'phone_exchange_id', 'phone_line_id'))

In [ ]:
df_pivot_pc = df_phone_contact.groupBy('customer_id').agg(collect_list('full_phone').alias('last_3_changes_list'))

In [ ]:
df_pivot_phone = df_pivot_pc.selectExpr('customer_id', 'last_3_changes_list[0] as phone_1', 'last_3_changes_list[1] as phone_2', 'last_3_changes_list[2] as phone_3')

In [ ]:
df_pivot_phone = df_pivot_phone.na.fill('---')
df_pivot_phone.show()

+-----------+---------------+---------------+-------+
|customer_id|        phone_1|        phone_2|phone_3|
+-----------+---------------+---------------+-------+
|   00221785|54-11-3696-6021|            ---|    ---|
|   00585832|54-11-5339-4383|            ---|    ---|
|   00857998|54-388-409-9742|            ---|    ---|
|   00890271|54-11-2492-4876|            ---|    ---|
|   01035263|54-294-425-7373|            ---|    ---|
|   01042668|54-11-2295-2413|            ---|    ---|
|   01136850|54-11-6959-8581|            ---|    ---|
|   01508497|54-11-3868-8489|            ---|    ---|
|   01552941|54-11-3104-0391|            ---|    ---|
|   01554790|54-11-3096-4113|            ---|    ---|
|   01867148|54-263-444-9082|            ---|    ---|
|   01944559|54-3462-23-4113|54-3462-42-2372|    ---|
|   01967286|54-11-5961-9424|54-11-5329-3089|    ---|
|   02394468|54-221-461-6821|54-11-6513-6425|    ---|
|   02409159|54-336-434-6906|            ---|    ---|
|   02425744|54-261-255-1104

####- 28 - Filtrar el DataFrame de direcciones de clientes y resguardar los 3 contactos más actuales por cliente.

In [ ]:
df_customer_address_sorted = df_customer_address.orderBy([df_customer_address.customer_id, desc('last_change_date')])

In [ ]:
window = Window.partitionBy(df_customer_address_sorted.customer_id).orderBy(desc(df_customer_address_sorted.last_change_date))

In [ ]:
df_address_contact = df_customer_address_sorted.withColumn('row_num', row_number().over(window))
df_address_contact = df_address_contact.filter(df_address_contact.row_num <= 3)

In [ ]:
df_address_contact = df_address_contact.withColumn("full_address", concat('street_name', 'address_outdoor_id', 'address_indoor_id', 'indoor_number', 'address_department_name', 'province_id', 'zipcode_id'))

In [ ]:
df_pivot_ad = df_address_contact.groupBy('customer_id').agg(collect_list('full_address').alias('last_3_changes_list'))

In [ ]:
df_pivot_address = df_pivot_ad.selectExpr('customer_id', 'last_3_changes_list[0] as address_1', 'last_3_changes_list[1] as address_2', 'last_3_changes_list[2] as address_3')

In [ ]:
df_pivot_address = df_pivot_address.na.fill('---')
df_pivot_address.toPandas()

,customer_id,address_1,address_2,address_3
0,00000442,AGUERO 0000842PB B ...,---,---
1,00001419,ALTE G BROWN 00010741 A ...,---,---
2,00001939,JOSE HERNANDEZ 00020451 C ...,---,---
3,00002707,GUATEMALA 0001846 ...,---,---
4,00002790,AV CORDOBA 00022064 30 ...,---,---
...,...,...,...,...
1206,29015563,IBANEZ TENIENTE 1 RO 0001355 ...,---,---
1207,29015902,GARIBALDI 0001554 ...,---,---
1208,29017100,DOMINGO MATHEU 0000972 ...,---,---
1209,29017191,VIRREY ARREDONDO 00026413 B ...,---,---


####- 29 - Filtrar el DataFrame de correos electrónicos de clientes  y resguardar los 3 contactos más actuales por cliente. 

In [ ]:
df_customer_emails_sorted = df_customer_emails.orderBy([df_customer_emails.customer_id, desc('last_change_date')])

In [ ]:
window = Window.partitionBy(df_customer_emails_sorted.customer_id).orderBy(desc(df_customer_emails_sorted.last_change_date))

In [ ]:
df_email_contact = df_customer_emails_sorted.withColumn('row_num', row_number().over(window))
df_email_contact = df_email_contact.filter(df_email_contact.row_num <= 3)

In [ ]:
df_pivot_ec = df_email_contact.groupBy('customer_id').agg(collect_list('email_desc').alias('last_3_changes_list'))

In [ ]:
df_pivot_email = df_pivot_ec.selectExpr('customer_id', 'last_3_changes_list[0] as email_1', 'last_3_changes_list[1] as email_2', 'last_3_changes_list[2] as email_3')

In [ ]:
df_pivot_email = df_pivot_email.na.fill('---')
df_pivot_email.toPandas()

,customer_id,email_1,email_2,email_3
0,00001419,JU_LY1@HOTMAIL.COM,EMILIA.RUBIANES@HOTMAIL.COM,---
1,00002790,NOTIENE@HOIMAIL.COM,DIGITALIZACION@EECC.COM,---
2,00014664,alq@ciudad.com.ar,---,---
3,00056407,NELLY.S.GEREZ@GMAIL.COM,---,---
4,00058801,SOPLETESHLANDA@HOTMAIL.COM,SOPLETESHLANDA@HOTMAIL.COM,---
...,...,...,...,...
317,28990339,ELSAMO@GMAIL.COM,---,---
318,28993945,navyig@fibertel.com.ar,---,---
319,29003190,ANLAU_08@LIVE.COM.AR,---,---
320,29008648,ROBERTOWINY@GMAIL.COM,---,---


# Enrequecimiento de datos

####- 30 - Agregar una nueva columna a los DataFrame de contactos, indicando el contact_type según corresponda (address, email, phone)

In [ ]:
df_phones_contact_col = df_phones.withColumn('contact_type_phones', lit('phone'))
df_phones_contact_col.select('customer_id','contact_type_phones').show()

+-----------+-------------------+
|customer_id|contact_type_phones|
+-----------+-------------------+
|   29377168|              phone|
|   29385404|              phone|
|   29385404|              phone|
|   29385454|              phone|
|   27626595|              phone|
|   24349744|              phone|
|   29389994|              phone|
|   29389999|              phone|
|   01192665|              phone|
|   29359207|              phone|
|   29354182|              phone|
|   22538341|              phone|
|   29387551|              phone|
|   29387551|              phone|
|   29353051|              phone|
|   29132899|              phone|
|   29390010|              phone|
|   27617687|              phone|
|   27617687|              phone|
|   29387672|              phone|
+-----------+-------------------+
only showing top 20 rows



In [ ]:
df_emails_contact_col = df_emails.withColumn('contact_type_emails', lit('e-mail'))
df_emails_contact_col.select('customer_id','contact_type_emails').show()

+-----------+-------------------+
|customer_id|contact_type_emails|
+-----------+-------------------+
|   00038638|             e-mail|
|   00037083|             e-mail|
|   06179762|             e-mail|
|   06179762|             e-mail|
|   26957165|             e-mail|
|   01128123|             e-mail|
|   28886249|             e-mail|
|   27514348|             e-mail|
|   27333422|             e-mail|
|   00031275|             e-mail|
|   21938599|             e-mail|
|   28990339|             e-mail|
|   00033622|             e-mail|
|   00033991|             e-mail|
|   03692218|             e-mail|
|   08698875|             e-mail|
|   27335212|             e-mail|
|   00038389|             e-mail|
|   00043705|             e-mail|
|   00043705|             e-mail|
+-----------+-------------------+
only showing top 20 rows



In [ ]:
df_address_contact_col = df_address.withColumn('contact_type_address', lit('address'))
df_address_contact_col.select('customer_id','contact_type_address').show()

+-----------+--------------------+
|customer_id|contact_type_address|
+-----------+--------------------+
|   00000660|             address|
|   07121078|             address|
|   22374047|             address|
|   00002450|             address|
|   00003925|             address|
|   00004038|             address|
|   01188067|             address|
|   00006509|             address|
|   00006754|             address|
|   00009582|             address|
|   26761763|             address|
|   00012730|             address|
|   23483616|             address|
|   00051576|             address|
|   00016049|             address|
|   00016995|             address|
|   26074313|             address|
|   27666452|             address|
|   00001213|             address|
|   00001213|             address|
+-----------+--------------------+
only showing top 20 rows



####- 31 - Agregar una nueva columna al DataFrame de contactos telefónicos de clientes, resguardando el contacto en formato json contenido en string, con los datos: Phone_type (mobile, landline ), Código país., Código de Área, Número teléfono.

In [ ]:
df_phone_contact = df_phone_contact.withColumn("phone_contact", to_json(struct(df_phone_contact.phone_type, df_phone_contact.phone_country_id, df_phone_contact.prefix_phone_id, df_phone_contact.phone_area_id, df_phone_contact.cellphone_prefix_id, df_phone_contact.phone_exchange_id, df_phone_contact.phone_line_id)))

In [ ]:
df_phone_contact.show(truncate=False)

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+------------+---------------+-----------------------+------------------+---------------------+-----------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+---------------

####- 32 - Agregar una nueva columna al DataFrame de direcciones de clientes, resguardando el contacto en formato json contenido en string, con los datos: Calle, Número, Piso, Depto, Localidad, Provincia, Código postal

In [ ]:
df_address_contact = df_address_contact.withColumn("address_contact",to_json(struct(df_address_contact.street_name,df_address_contact.address_outdoor_id,df_address_contact.address_indoor_id,df_address_contact.indoor_number,df_address_contact.address_district_name, df_address_contact.address_town_name,df_address_contact.zipcode_id)))

In [ ]:
df_address_contact.show(truncate=False)

+-----------+-----------------------+-------------------+--------------+------------------------+------------------+---------------------------+-----------------+-------------+------------------------------+------------------------------+------------------------------+----------+---------------+-----------+------------------+------------------------------------------------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-------------

####- 33 - Combinar los DataFrame de contactos telefónicos de clientes, direcciones de clientes y email de clientes en uno solo.

In [ ]:
df_contacts = df_pivot_phone.join(df_pivot_email, "customer_id", how='full').join(df_pivot_address, "customer_id", how='full')

In [ ]:
df_contacts.toPandas()

,customer_id,phone_1,phone_2,phone_3,email_1,email_2,email_3,address_1,address_2,address_3
0,00000442,None,None,None,None,None,None,AGUERO 0000842PB B ...,---,---
1,00001419,None,None,None,JU_LY1@HOTMAIL.COM,EMILIA.RUBIANES@HOTMAIL.COM,---,ALTE G BROWN 00010741 A ...,---,---
2,00001939,None,None,None,None,None,None,JOSE HERNANDEZ 00020451 C ...,---,---
3,00002707,None,None,None,None,None,None,GUATEMALA 0001846 ...,---,---
4,00002790,None,None,None,NOTIENE@HOIMAIL.COM,DIGITALIZACION@EECC.COM,---,AV CORDOBA 00022064 30 ...,---,---
...,...,...,...,...,...,...,...,...,...,...
1607,30965114,54-0011-3242-8160,---,---,None,None,None,None,None,None
1608,30970953,54-11-2292-3679,---,---,None,None,None,None,None,None
1609,30973215,54-291-513-7482,---,---,None,None,None,None,None,None
1610,30973892,54-11-3757-3423,---,---,None,None,None,None,None,None


In [ ]:
df_short_contacts = df_contacts.select('customer_id','Phone_1','Email_1','Address_1')
df_short_contacts.toPandas()

,customer_id,Phone_1,Email_1,Address_1
0,00000442,None,None,AGUERO 0000842PB B ...
1,00001419,None,JU_LY1@HOTMAIL.COM,ALTE G BROWN 00010741 A ...
2,00001939,None,None,JOSE HERNANDEZ 00020451 C ...
3,00002707,None,None,GUATEMALA 0001846 ...
4,00002790,None,NOTIENE@HOIMAIL.COM,AV CORDOBA 00022064 30 ...
...,...,...,...,...
1607,30965114,54-0011-3242-8160,None,None
1608,30970953,54-11-2292-3679,None,None
1609,30973215,54-291-513-7482,None,None
1610,30973892,54-11-3757-3423,None,None


----

In [ ]:
df_contact_types = df_phones_contact_col.join(df_address_contact_col, 'customer_id', how='full').join(df_emails_contact_col, 'customer_id', how='full')

In [ ]:
df_contact_types.select('customer_id','contact_type_phones','contact_type_address','contact_type_emails').toPandas()

,customer_id,contact_type_phones,contact_type_address,contact_type_emails
0,00000007,phone,None,None
1,00000188,None,address,None
2,00000204,None,address,None
3,00000228,None,address,None
4,00000274,None,address,None
...,...,...,...,...
432414,30976146,phone,None,None
432415,30976148,phone,None,None
432416,30976151,phone,None,None
432417,30976154,phone,None,None


# Creacion de vistas temporales

####- 35 - Generar una vista temporal a partir del DataFrame de contactos.

In [ ]:
df_short_contacts.createTempView('tw_contacts')

AnalysisException: ignored

####- 36 - Generar una vista temporal a partir del archivo t_abtq_customer_basics.

In [ ]:
df_customer.createOrReplaceTempView('tw_customer_basics')

####- 37 - Generar una vista temporal a partir del archivo t_acog_marital_status_type.

In [ ]:
df_marital_status_ok.createOrReplaceTempView('tw_marital_status')

####- 38 - Generar una vista temporal a partir del archivo t_acog_nationality.

In [ ]:
df_nationality.createOrReplaceTempView("tw_nationality")

# Querys en spark SQL.

####- 39 - Generar un público objetivo (1) que cumpla los siguientes puntos:
* Cliente
* Antigüedad superior a 5 años
* Asalariado Fijo.
* Estado civil: Casado


In [ ]:
publico_obj1 = sqlContext.sql('SELECT cb.customer_id, floor(datediff(current_date(), admission_date)/365) as years_diff, customer_condition_type, cb.job_type_desc, ms.martial_status_short_desc FROM tw_customer_basics as cb INNER JOIN tw_marital_status as ms ON cb.marital_status_type = ms.marital_status_type WHERE floor(datediff(current_date(), admission_date)/365) > 5 AND cb.marital_status_type = 01 AND cb.job_type = 001 AND cb.customer_condition_type = 1')

In [ ]:
publico_obj1.toPandas()

####- 40 -  Generar un público objetivo (2) que cumpla los siguientes puntos:
* Potencial Cliente
* Sexo Femenino
* Entre 30 y 45 años de Edad.
* Nacionalidad No Argentina

In [ ]:
publico_obj2 = sqlContext.sql('SELECT cb.customer_id, customer_condition_type, gender_type, floor(datediff(current_date(), birth_date)/365) as customer_age, n.country_name FROM tw_customer_basics AS cb INNER JOIN tw_nationality AS n ON cb.country_nationality_id == n.country_nationality_id WHERE floor(datediff(current_date(), birth_date)/365) > 30 AND floor(datediff(current_date(), birth_date)/365) < 45 AND gender_type = "F" AND customer_condition_type = 1 AND n.country_nationality_id <> 80')
publico_obj2.toPandas()

####- 41 - Generar un público objetivo (3) que cumpla los siguientes puntos:
* Potencial Cliente
* Sexo Masculino.
* Mayor a 25 años.
* Estado civil Soltero

In [ ]:
publico_obj3 = sqlContext.sql('SELECT cb.customer_id, customer_condition_type, gender_type, floor(datediff(current_date(), birth_date)/365) as customer_age, ms.martial_status_short_desc FROM tw_customer_basics AS cb INNER JOIN tw_marital_status as ms ON cb.marital_status_type = ms.marital_status_type WHERE customer_condition_type = 1 AND floor(datediff(current_date(), birth_date)/365) > 25 AND gender_type = "M" AND cb.marital_status_type = 5')
publico_obj3.toPandas()

####- 42 - Generar un público objetivo (4) que cumpla los siguientes puntos:
* Cliente
* Edad superior a 52 años
* Nacionalidad Argentina
* Estado civil Viudo

In [ ]:
publico_obj4 = sqlContext.sql('SELECT cb.customer_id, floor(datediff(current_date(), birth_date)/365) as customer_age, n.country_name, ms.martial_status_short_desc FROM tw_customer_basics as cb INNER JOIN tw_nationality AS n ON cb.country_nationality_id == n.country_nationality_id INNER JOIN tw_marital_status as ms ON cb.marital_status_type = ms.marital_status_type WHERE floor(datediff(current_date(), birth_date)/365) > 52 AND cb.country_nationality_id = 80 AND cb.marital_status_type = 2')
publico_obj4.toPandas()

# Carga de datos, generacion de archivos .csv
#### Los archivos deben cumplir los siguientes requerimientos: 
* Contener cabecera. 
* Separador “|”. 
* Máximo de 1000 registros por archivo (si el público objetivo tiene más de 1000 registros, se deberá crear más de un archivo)
* Ser guardados en HDFS.

In [ ]:
# los dataframes no contienen más de 1000 registros, por lo que no es necesrio partir los .csv. Para partirlos, hubieramos necesitado las funciones 'count' y 'repartition'

#### - 43 -  Generar archivos .csv a partir de los público objetivo 1. 


In [ ]:
# primero cuento la cantidad de celdas para ver si es necesario partir el archivo
publico_obj1.count()

In [ ]:
# son 250 registros, por lo tanto no debo partir el data frame
# no lo exporto como HDFS porque no estoy usando un contenedor, por lo tanto no tengo un número de puerto
publico_obj1.write \
  .option("delimiter", "|") \
  .option("header", "true") \
  .csv("Datasets/P1")

#### - 44 - Generar archivos .csv a partir del público objetivo 2. 

In [ ]:
# primero cuento la cantidad de celdas para ver si es necesario partir el archivo
publico_obj2.count()

In [ ]:
# son 2 registros, por lo tanto no debo partir el data frame
# no lo exporto como HDFS porque no estoy usando un contenedor, por lo tanto no tengo un número de puerto
publico_obj2.write \
  .option("delimiter", "|") \
  .option("header", "true") \
  .csv("Datasets/P2")

####- 45 - Generar archivos .csv a partir del público objetivo 3. 

In [ ]:
# primero cuento la cantidad de celdas para ver si es necesario partir el archivo
publico_obj3.count()

In [ ]:
# son 299 registros, por lo tanto no debo partir el data frame
# no lo exporto como HDFS porque no estoy usando un contenedor, por lo tanto no tengo un número de puerto
publico_obj3.write \
  .option("delimiter", "|") \
  .option("header", "true") \
  .csv("Datasets/P3")

####- 46 - Generar archivos .csv a partir del público objetivo 4. 

In [ ]:
# primero cuento la cantidad de celdas para ver si es necesario partir el archivo
publico_obj4.count()

In [ ]:
# son 20 registros, por lo tanto no debo partir el data frame
# no lo exporto como HDFS porque no estoy usando un contenedor, por lo tanto no tengo un número de puerto
publico_obj4.write \
  .option("delimiter", "|") \
  .option("header", "true") \
  .csv("Datasets/P4")

# Extraccion de datos desde .csv, públicos objetivo

#### - 47 - Cargar los datos de los publicos generados en un DataFrame.

In [ ]:
df_pobj1csv = spark.read.format("csv") \
  .option("delimiter", "|") \
  .option("header", "false") \
  .load("/content/Datasets/P1/part-00000-b3b88a32-8951-4f8f-9429-f5e1684307b3-c000.csv")
  
df_pobj2csv = spark.read.format("csv") \
  .option("delimiter", "|") \
  .option("header", "false") \
  .load("/content/Datasets/P2/part-00000-dc179214-a403-45ee-a865-e0791719467f-c000.csv")
  
df_pobj3csv = spark.read.format("csv") \
  .option("delimiter", "|") \
  .option("header", "false") \
  .load("/content/Datasets/P3/part-00000-fd03bdfe-0656-421a-89da-2f72b5d4babe-c000.csv")

df_pobj4csv = spark.read.format("csv") \
  .option("delimiter", "|") \
  .option("header", "false") \
  .load("/content/Datasets/P4/part-00000-f4dd6ba1-39ab-49ee-a823-2502a1f59867-c000.csv")

# Calculo de agregaciones.

####- 48 -  Contar al cantidad de personas contactada por publico objetivo.

In [ ]:
df_pobj4csv.show()

In [ ]:
df_pobj1csv.select("_c0").distinct().count() - 1

In [ ]:
df_pobj2csv.select("_c0").distinct().count() - 1

In [ ]:
df_pobj3csv.select("_c0").distinct().count() - 1

In [ ]:
df_pobj4csv.select("_c0").distinct().count() - 1

####- 49 - Contar la cantidad de direcciones, mails y telefonos por publico objetivo.

####- 50 - Contar la cantidad de hombres y mujeres por publico objetivo.

#### - 51 - Calcular la edad promedio por publico objetivo.

# Carga de datos, generacion de archivos .parquet

####- 52 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

####- 53 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

####- 54 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

####- 55 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

# Borrador

In [ ]:
df_customer.show() # admission_date marital_status_type gender_type birth_date job_type profession_id
#df_phones.toPandas()
#df_address.toPandas()
#df_emails.toPandas()
#df_marital_status.toPandas() # marital_status_type martial_status_short_desc
#df_segment_type.toPandas() 
#df_customer_documents.toPandas() 
#df_address_type.toPandas() # address_type address_type_short_desc ??
#df_gender.toPandas() # gender_type gender_desc
#df_nationality.toPandas() # country_nationality_id # country_nationality_id
#df_personal_type.toPandas()
#df_phone_type.toPandas() # phone_type	phone_type_desc	 ??
#df_province.toPandas()
#df_customer_segment.show() 
#df_customer_info_temp.show()

# VER DE NO IMPORTAR PARQUETS QUE NO USEMOS